# A

In [ ]:
#!/usr/bin/env python

# coding: utf-8

# %% Loading Packages
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from collections import Counter
# from sklearn.metrics import mean_squared_error, mean_absolute_erIPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')ror

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import ReLU
from tensorflow.keras.layers import Dropout
from tensorflow.keras.activations import tanh
from tensorflow.keras.layers import PReLU
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.losses import MSE, MAE, MAPE
from tensorflow.keras.metrics import Accuracy, SparseCategoricalAccuracy
from tensorflow.keras.losses import kullback_leibler_divergence, SparseCategoricalCrossentropy, KLDivergence
from tensorflow.keras.callbacks import LearningRateScheduler

import sys

from tensorflow.python.ops.gen_batch_ops import batch

import sys
import os
import sys
import platform

from tensorflow.python.ops.gen_math_ops import Tanh
if platform.node() in ['msbq', 'msdai']:
    os.chdir('/home/ms/github/fxpred')
    sys.path.append(os.path.join(os.getcwd(), 'Transformer'))
from utils import get_fx_and_metric_data_wo_weekend, mde
from utils_NN_opt_learning_rate import opt_learn_rate_plot
from benchmark_utils import actual_pred_plot, ts_train_test_normalize
from transformer_architecture_elements import Time2Vector, SingleAttention, MultiAttention, TransformerEncoder

In [ ]:
# filename = 'Benchmark/trained_models/LSTM_univariate2_classification____0100.hdf5'  # Classification model
# filename = 'Benchmark/trained_models/LSTM_univariate2_with_close_price____0150.hdf5'  # Regression model
# filename = 'content/Transformer+TimeEmbedding2_minmax____0300.hdf5'
filename = 'content/FXTransformerMulti0001.hdf5'
# filename = 'content/Transformer+TimeEmbeddingClassification0010.hdf5'

my_LSTM_model = tf.keras.models.load_model(
    filename,
    custom_objects={
        'kl_divergence': kullback_leibler_divergence,
        'Time2Vector': Time2Vector,
        'SingleAttention': SingleAttention,
        'MultiAttention': MultiAttention,
        'TransformerEncoder': TransformerEncoder
    }
)
lag = my_LSTM_model.input_shape[1]
print(f'lag: {lag}')
if my_LSTM_model.layers[-1].activation == tf.keras.activations.softmax:
    nn_type_classification = True
    h = 1
else:
    nn_type_classification = False
    h = my_LSTM_model.output_shape[1]
print(f'h: {h}')

In [ ]:
def predictions(my_model, X_test, sc=None):
    LSTM_prediction = my_model.predict(X_test)
    if sc is not None:
        LSTM_prediction = sc.inverse_transform(LSTM_prediction)
    return LSTM_prediction

In [ ]:
dtype = np.float32  # np.float64
tf.keras.backend.set_floatx('float32')

df = get_fx_and_metric_data_wo_weekend(dtype=dtype)
target_price = target = 'EURUSD BGNE Curncy Bid Close'
df_close = get_fx_and_metric_data_wo_weekend(dtype=dtype, pct_change=False)
target_column = list(df.columns).index(target)
if my_LSTM_model.input_shape[-1] <= 2:
    df = df.iloc[:, target_column : target_column + 1]
target_column = list(df.columns).index(target)
target_price_column = target_column
print(df.shape)
df.head()

In [ ]:
def strategy(x):
    conditions = [
        (x <= -0.00000001),
        (x > -0.00000001) & (x <= 0.00000001),
        (x > 0.00000001)
    ]

    # create a list of the values we want to assign for each condition #!!! 
    values = ['sell', 'hold', 'buy']
    values = [0, 1, 2]

    # create a new column and use np.select to assign values to it using our lists as arguments #!!! 
    return np.select(conditions, values).astype(np.int32)
df['strategy'] = strategy(df[target_price])

if nn_type_classification:    
    target = 'strategy'
target_column = df.columns.to_list().index(target)

In [ ]:
df[target_price].plot()
plt.title('Return of closing price')
plt.show()

In [ ]:
X_train, y_train, X_val, y_val , X_test, y_test, sc, sc_target, index_train, index_val, index_test, X_train_index, y_train_index = \
    ts_train_test_normalize(df, lag, h, target_column)

In [ ]:
if nn_type_classification:
    y_train = sc_target.inverse_transform(y_train)
    y_val = sc_target.inverse_transform(y_val)
    y_test = sc_target.inverse_transform(y_test)

## Econmetric measures

Definition of cumulated returns

In [ ]:
def cumulated_return(y_pred, ret, start_capital=1000, dtype=None):
    assert y_pred.flatten().shape == ret.flatten().shape
    balance = [start_capital]
    signal = 0
    for i, d in enumerate(y_pred[:-1]):
        signal = signal if d == 1 else d
        if signal == 0:
            balance.append(balance[-1])
        elif signal == 2:
            balance.append(balance[-1] * ret[i])
        else:
            raise ValueError()
    balance = np.array(balance, dtype=dtype)
    return balance

In [ ]:
#!!! for testing purposes
tmp_type = 'train'

X = np.concatenate([sc.inverse_transform(i)[np.newaxis] for i in eval(f"X_{tmp_type}")], axis=0)
for i in range(lag):
    diff = np.abs(df.loc[eval(f"index_{tmp_type}")].iloc[i:-(lag - i + 1)].values - X[:,-(lag - i)]).sum(axis=0)
    assert diff[0] < 1e-6 and diff[1] < 0.5, 'df and X inversed are not the same'

X_ret_testing = np.concatenate([sc.inverse_transform(i)[np.newaxis] for i in eval(f"X_{tmp_type}")], axis=0)[:, -1, 0]
y_testing = np.concatenate([sc.inverse_transform(i)[np.newaxis] for i in eval(f"X_{tmp_type}")], axis=0)[:, -1, 1]

a1 = sc_target.inverse_transform(eval(f"y_{tmp_type}")).flatten()
a2 = eval(f"y_{tmp_type}").flatten()
b = np.concatenate([sc.inverse_transform(i)[np.newaxis] for i in eval(f"X_{tmp_type}")], axis=0)[:, -1, 1]
cc = df.loc[eval(f"index_{tmp_type}")].iloc[(lag - 1):-(h+1), 1].values
print(np.abs(cc[1:] - a1[:-1]).sum(axis=0))
print(np.abs(cc[1:] - a2[:-1]).sum(axis=0))
print(np.abs(cc - b).sum(axis=0))

In [ ]:
print((index_train[0:-(lag+1)] == X_train_index[:, 0]).all())
print((index_train[(lag-1):-(h+1)] == X_train_index[:, -1]).all())

## read in data for econmic metric

In [ ]:
def econ_metrics(y_pred, X_ret, risk_free=0.0031):  # Risk free rate must be given
    x = cumulated_return(y_pred[:-1], X_ret[1:], dtype=dtype)
    print(f'Portfolio return: {x[-1] / x[0] - 1}')
    print(f'Std of returns: {np.std(x[1:] / x[:-1])}')
    print(f'Sharpe ratio: {((x[-1] / x[0]) - 1 - risk_free) / np.std(x[1:] / x[:-1])}')
    print()
    
    n = 6 * 24 * 252  # banking days per year
    mu_annum = ((x[-1]/x[0]) ** (n / x.shape[0])) - 1
    print(f'Return per annum: {mu_annum}')
    std_annum = np.std(x[1:] / x[:-1]) * (n)**0.5
    print(f'Std of returns per annum: {std_annum}')
    print(f'Sharpe ratio per annum: {(mu_annum - risk_free) / std_annum}')

In [ ]:
X_train.shape[-1] == my_LSTM_model.input_shape[-1]
print(X_train.shape[-1])
print(my_LSTM_model.input_shape[-1])

In [ ]:
def result_output2(tmp_type):
    print(tmp_type)

    y = eval(f"y_{tmp_type}")
    if nn_type_classification:
        y_pred = np.argmax(predictions(
            my_LSTM_model,
            eval(f"X_{tmp_type}")
        ), axis=1)
    else:
        y = strategy(y)
        if X_train.shape[-1] - 1 == my_LSTM_model.input_shape[-1]:
            i = df.columns.to_list().index('strategy')
            y_pred = strategy(predictions(
                my_LSTM_model,
                np.delete(eval(f"X_{tmp_type}"), i, -1)
            ))
        else:
            y_pred = strategy(predictions(
                my_LSTM_model,
                eval(f"X_{tmp_type}")
            ))
    print(f'Accuracy: {Accuracy()(y.flatten(), y_pred.flatten()).numpy()}')
    print(f'Accuracy: {SparseCategoricalAccuracy()(y.astype(dtype), y_pred.reshape(y.shape).astype(dtype)).numpy()}')
    print(f'KL_Divergence: {kullback_leibler_divergence(y.flatten().astype(dtype), y_pred.flatten().astype(dtype)).numpy()}')
    print(classification_report(y, y_pred))
    cm = confusion_matrix(y, y_pred)
    ConfusionMatrixDisplay(cm).plot()
    plt.show()

    X_ret = sc.inverse_transform(eval(f"X_{tmp_type}"))[:, -1, target_price_column] + 1
    plt.plot(eval(f"index_{tmp_type}")[lag-1:-(h+1)][1:], cumulated_return(y_pred[:-1], X_ret[1:], dtype=dtype), label='predicted')

    c = Counter(y.flatten())
    for i in range(3):
        c.update({i: 0})
    p = [i[1]/y.shape[0] for i in sorted(c.items(), key=lambda x: x[0])]
    x = np.concatenate(
        [
            cumulated_return(
                np.random.choice([0, 1, 2], size=y_pred[:-1].shape, p=p),
                X_ret[1:],
                dtype=dtype
            )[:,np.newaxis] for i in range(50)
        ], axis=-1
    )
    plt.plot(eval(f"index_{tmp_type}")[(lag-1):-(h+1)][1:], x.mean(axis=-1), label='random', color='gray')
    plt.plot(eval(f"index_{tmp_type}")[(lag-1):-(h+1)][1:], x.mean(axis=-1) + x.std(axis=-1), label='random_q75', color='gray', linestyle='--')
    plt.plot(eval(f"index_{tmp_type}")[(lag-1):-(h+1)][1:], cumulated_return(y_pred[:-1] * 0 + 2, X_ret[1:], dtype=dtype), label='buy&HODL')
    plt.plot(eval(f"index_{tmp_type}")[(lag-1):-(h+1)][1:], cumulated_return(y[:-1], X_ret[1:], dtype=dtype), label='oracle')
    plt.plot(eval(f"index_{tmp_type}")[(lag-1):-(h+1)][1:], cumulated_return(y, X_ret, dtype=dtype)[:-1], label='oracle')
    plt.legend()
    plt.semilogy()
    plt.show()
    econ_metrics(y_pred, X_ret)

In [ ]:
result_output2('train')

In [ ]:
result_output2('val')

In [ ]:
result_output2('test')